In [1]:
'''Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
'''
from __future__ import print_function

import pandas as pd
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants

import mlflow
import mlflow.tensorflow
from mlflow import pyfunc

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

# Setup Experiment Tracker

In [3]:
tracking_uri='file:///mnt/pipelineai/users/experiments'
mlflow.set_tracking_uri(tracking_uri)

experiment_name = 'boston'
mlflow.set_experiment(experiment_name)  

import mlflow.tensorflow
mlflow.tensorflow.autolog()

# Import Training Data 

In [4]:
# Builds, trains and evaluates a tf.estimator. Then, exports it for inference, logs the exported model
# with MLflow, and loads the fitted model back as a PyFunc to make predictions.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data()
# There are 13 features we are using for inference.
feat_cols = [tf.feature_column.numeric_column(key="features", shape=(x_train.shape[1],))]
feat_spec = {
    "features": tf.placeholder("float", name="features", shape=[None, x_train.shape[1]])}
hidden_units = [50, 20]
steps = 1000
regressor = tf.estimator.DNNRegressor(hidden_units=hidden_units, feature_columns=feat_cols)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"features": x_train}, y_train,
                                                    num_epochs=None, shuffle=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgtxi9wir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9d557bcc18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Start Training Run

In [5]:
with mlflow.start_run() as run:
    mlflow.log_param("Hidden Units", hidden_units)
    mlflow.log_param("Steps", steps)
    regressor.train(train_input_fn, steps=steps)
    test_input_fn = tf.estimator.inputs.numpy_input_fn({"features": x_test}, y_test,
                                                       num_epochs=None, shuffle=True)
    # Compute mean squared error
    mse = regressor.evaluate(test_input_fn, steps=steps)
    mlflow.log_metric("Mean Square Error", mse['average_loss'])
    # Building a receiver function for exporting
    receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feat_spec)
    
    saved_estimator_path = regressor.export_savedmodel('./saved-models/boston', receiver_fn).decode("utf-8")
    # Logging the saved model
    mlflow.tensorflow.log_model(tf_saved_model_dir=saved_estimator_path,
                                tf_meta_graph_tags=[tag_constants.SERVING],
                                tf_signature_def_key="predict",
                                artifact_path="model")    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpgtxi9wir/model.ckpt.
INFO:tensorflow:loss = 106129.97, step = 1
INFO:tensorflow:global_step/sec: 344.258
INFO:tensorflow:loss = 11762.465, step = 101 (0.229 sec)
INFO:tensorflow:global_step/sec: 512.569
INFO:tensorflow:loss = 5105.897, step = 201 (0.387 sec)
INFO:tensorflow:global_step/sec: 245.903
INFO:tensorflow:loss = 4354.6533, step =

/opt/conda/lib/python3.6/site-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
2019/08/19 16:57:04 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ./saved-models/boston/1566233824/variables/variables


2019/08/19 16:57:04 INFO mlflow.tensorflow: Validation succeeded!
2019/08/19 16:57:04 INFO mlflow.tensorflow: Validating the specified TensorFlow model by attempting to load it in a new TensorFlow graph...


INFO:tensorflow:Restoring parameters from ./saved-models/boston/1566233824/variables/variables


2019/08/19 16:57:05 INFO mlflow.tensorflow: Validation succeeded!


# Predict with the Model

In [ ]:
# Resume the run by passing run id from above
with mlflow.start_run(run_id=run.info.run_id) as run:
    # Reload the model and predict
    pyfunc_model = mlflow.pyfunc.load_model(mlflow.get_artifact_uri('model'))

    # Predict with the loaded Python Function
    df = pd.DataFrame(data=x_test, columns=["features"] * x_train.shape[1])
    predict_df = pyfunc_model.predict(df)
    predict_df['original_labels'] = y_test
    print(predict_df)

# Check the MLflow Pipelines Tab
![MLflow Pipeline](https://raw.githubusercontent.com/PipelineAI/site/master/assets/img/mlflow-pipeline3.png)